In [ ]:
from dotenv import load_dotenv
import awswrangler as wr
import os
from pathlib import Path
import boto3

if "notebook" in os.getcwd():
    os.chdir("..")

load_dotenv()

In [ ]:
client = boto3.client('sts')
assume_role_response = client.assume_role(
    RoleArn=os.environ["role_arn"],
    RoleSessionName="test-read-access",
)
sts_credentials = assume_role_response["Credentials"]

In [ ]:
boto3_session = boto3.Session(
    aws_access_key_id=sts_credentials["AccessKeyId"],
    aws_secret_access_key=sts_credentials["SecretAccessKey"],
    aws_session_token=sts_credentials["SessionToken"],
    region_name="us-east-1"
)

s3_client = boto3_session.client("s3")

In [ ]:
# arbitrarily select an object from the bucket to test access
epa_bucket = os.environ['epa_bucket']
response = s3_client.list_objects_v2(Bucket=epa_bucket)

In [ ]:
# test copying over an object
object_key = response['Contents'][1]['Key']

copy_response = wr.s3.copy_objects(
    paths=[f"s3://{epa_bucket}/{object_key}"],
    source_path=f"s3://{epa_bucket}/{str(Path(object_key).parent)}/",
    target_path="s3://mtsu-test-bucket-sample/audio-files/",
    boto3_session=boto3_session
)